<a href="https://colab.research.google.com/github/engineering2940/VRP/blob/main/VRP_Indonesia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
## for data
import pandas as pd
import numpy as np
import os

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import plotly.express as px

In [5]:
!pip install osmnx==1.3.0

In [6]:
## for simple routing
import osmnx as ox
import networkx as nx
import pickle  # Untuk menyimpan graph ke file

In [7]:
!pip install ortools

In [8]:
## for advanced routing
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

## **JAWA BARAT**

In [12]:
# Load dataset
file_path = "/content/k_gudang02.csv"
df = pd.read_csv(file_path)

# Filter hanya lokasi di Pulau Jawa
provinsi_jabar = ["JABAR"]
df_jabar = df[df["PROVINSI"].str.upper().isin(provinsi_jawa)].reset_index(drop=True)

# Cek jumlah lokasi yang tersedia
print(f"Total lokasi di Pulau Jawa: {len(df_jawa)}")
df_jabar.head()

Total lokasi di Pulau Jawa: 77


,NEGARA,PROVINSI,UNIT,LONGITUDE,LATITUDE
0,INDONESIA,JABAR,TANAH GUDANG CISARANTEN KIDUL,107.692716,-6.939044
1,INDONESIA,JABAR,TANAH I GUDANG UTAMA I,107.536566,-6.903000
2,INDONESIA,JABAR,TANAH II GUDANG UTAMA I,107.536566,-6.903000
3,INDONESIA,JABAR,TANAH III GUDANG UTAMA I,107.536566,-6.903000
4,INDONESIA,JABAR,TANAH IV GUDANG UTAMA I,107.536566,-6.903000


In [10]:
# Tentukan path penyimpanan
graph_path = "/content/graph_jabar.graphml"

# Periksa apakah graph sudah ada
if os.path.exists(graph_path):
    print("Memuat graph dari file...")
    G = ox.load_graphml(graph_path)
else:
    print("Mengunduh graph dari OSM...")
    place_name = "Jawa Barat, Indonesia"
    G = ox.graph_from_place(place_name, network_type="drive")
    ox.save_graphml(G, graph_path)
    print(f"Graph disimpan di {graph_path}")

print(f"Graph Jawa Barat berhasil dibuat dengan {len(G.nodes)} nodes dan {len(G.edges)} edges.")

Memuat graph dari file...
Graph Jawa Barat berhasil dibuat dengan 872589 nodes dan 2094094 edges.


In [13]:
# Konversi lokasi ke node di graph OSM
nodes = [ox.distance.nearest_nodes(G, row["LONGITUDE"], row["LATITUDE"]) for _, row in df_jabar.iterrows()]

# Buat Matriks Jarak berdasarkan Shortest Path di Graph OSM
def create_shortest_path_matrix(G, nodes):
    num_nodes = len(nodes)
    distance_matrix = np.zeros((num_nodes, num_nodes))

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                try:
                    # Gunakan Dijkstra untuk menghitung jarak jalan sebenarnya
                    distance_matrix[i][j] = nx.shortest_path_length(G, nodes[i], nodes[j], weight='length')
                except nx.NetworkXNoPath:
                    distance_matrix[i][j] = np.inf  # Jika tidak ada jalur, set ke infinity

    return distance_matrix

# Buat Matriks Jarak
distance_matrix = create_shortest_path_matrix(G, nodes)

print("Matriks jarak berdasarkan Dijkstra Shortest Path berhasil dibuat.")

Matriks jarak berdasarkan Dijkstra Shortest Path berhasil dibuat.


In [14]:
# Set parameter VRP
drivers = 10  # Jumlah kendaraan
driver_capacities = [10000] * drivers  # Kapasitas per kendaraan (kg)
max_distance = 300 * 1000  # 300 km dalam meter (Jawa Barat lebih kecil)

def solve_vrp(distance_matrix, max_distance, drivers, driver_capacities):
    num_nodes = len(distance_matrix)

    manager = pywrapcp.RoutingIndexManager(num_nodes, drivers, 0)
    model = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(distance_matrix[from_node][to_node])

    transit_callback_index = model.RegisterTransitCallback(distance_callback)
    model.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    model.AddDimension(transit_callback_index, 0, max_distance, True, "Distance")

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    solution = model.SolveWithParameters(search_parameters)

    if solution:
        routes = {}
        for driver in range(drivers):
            index = model.Start(driver)
            route = []
            while not model.IsEnd(index):
                node_index = manager.IndexToNode(index)
                route.append(nodes[node_index])
                index = solution.Value(model.NextVar(index))
            route.append(nodes[manager.IndexToNode(index)])
            routes[driver] = route
        return routes
    else:
        print("Tidak ada solusi VRP ditemukan.")
        return None

# Jalankan model VRP dengan Shortest Path
vrp_routes = solve_vrp(distance_matrix, max_distance, drivers, driver_capacities)

# Simpan hasil rute di memori Google Colab
if vrp_routes:
    print("VRP berhasil dijalankan untuk Jawa Barat!")

VRP berhasil dijalankan untuk Jawa Barat!


In [15]:
# Buat peta awal di Jawa Barat
m = folium.Map(location=[-6.9, 107.6], zoom_start=9)

# Tambahkan rute ke peta
colors = ["red", "blue", "green", "purple", "orange", "darkred", "lightblue"]
for i, (driver, route) in enumerate(vrp_routes.items()):
    color = colors[i % len(colors)]  # Pilih warna dari daftar
    points = [(G.nodes[n]["y"], G.nodes[n]["x"]) for n in route]
    folium.PolyLine(points, color=color, weight=2.5, opacity=1).add_to(m)

# Tambahkan marker untuk lokasi utama
for _, row in df_jabar.iterrows():
    folium.Marker(
        location=[row["LATITUDE"], row["LONGITUDE"]],
        popup=row["UNIT"],
        icon=folium.Icon(color="blue", icon="cloud")
    ).add_to(m)

# Simpan dan tampilkan peta
map_path = "/content/vrp_map_jabar.html"
m.save(map_path)
print(f"Peta VRP untuk Jawa Barat disimpan sebagai `{map_path}`.")

Peta VRP untuk Jawa Barat disimpan sebagai `/content/vrp_map_jabar.html`.
